In [1]:
import pandas as pd
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from llm_client_2 import generate_extra_meta


/home/cxiang/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-26 20:16:24 [__init__.py:256] Automatically detected platform cuda.


2025-03-26 20:16:24,891	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 03-26 20:16:33 [config.py:583] This model supports multiple tasks: {'classify', 'score', 'embed', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 03-26 20:16:33 [config.py:1693] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 03-26 20:16:35 [core.py:53] Initializing a V1 LLM engine (v0.8.1) with config: model='qwen/Qwen2.5-7B-Instruct', speculative_config=None, tokenizer='qwen/Qwen2.5-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=Non

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.64it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.65it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.63it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.67it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.66it/s]



INFO 03-26 20:16:40 [loader.py:429] Loading weights took 2.48 seconds
INFO 03-26 20:16:41 [gpu_model_runner.py:1176] Model loading took 14.2487 GB and 4.494886 seconds
INFO 03-26 20:16:46 [backends.py:409] Using cache directory: /home/cxiang/.cache/vllm/torch_compile_cache/cb05eb8a21/rank_0_0 for vLLM's torch.compile
INFO 03-26 20:16:46 [backends.py:419] Dynamo bytecode transform time: 5.01 s
INFO 03-26 20:16:46 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 03-26 20:16:50 [monitor.py:33] torch.compile takes 5.01 s in total
INFO 03-26 20:16:51 [kv_cache_utils.py:537] GPU KV cache size: 369,264 tokens
INFO 03-26 20:16:51 [kv_cache_utils.py:540] Maximum concurrency for 32,768 tokens per request: 11.27x
INFO 03-26 20:17:12 [gpu_model_runner.py:1499] Graph capturing finished in 20 secs, took 0.48 GiB
INFO 03-26 20:17:12 [core.py:138] init engine (profile, create kv cache, warmup model) took 31.04 seconds


Exception in thread Thread-7 (process_input_socket):
Traceback (most recent call last):
  File "/home/cxiang/miniconda3/envs/llm/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/cxiang/miniconda3/envs/llm/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/cxiang/miniconda3/envs/llm/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cxiang/miniconda3/envs/llm/lib/python3.10/site-packages/vllm/v1/engine/core.py", line 419, in process_input_socket
    type_frame, data_frame = socket.recv_multipart(copy=False)
ValueError: too many values to unpack (expected 2)


In [5]:
df = pd.read_json("../../judgements/eng_cases.json")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100753 entries, 0 to 100752
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   court                 100753 non-null  object        
 1   registry              100753 non-null  object        
 2   date                  100753 non-null  datetime64[ns]
 3   case_name             100753 non-null  object        
 4   citation              100753 non-null  object        
 5   url                   100753 non-null  object        
 6   case_number           100753 non-null  object        
 7   law_report_citations  100753 non-null  object        
 8   content               100753 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 6.9+ MB


In [ ]:
# Count entries for each court
court_counts = df['court'].value_counts()

# Display the counts
print("Number of cases by court:")
print(court_counts)

In [ ]:
# Count tokens in all content using a simple tokenization approach
total_tokens = 0

# Using a simple approach: space-based tokenization for a rough estimate
for content in df['content']:
    # Split by whitespace to get approximate token count
    tokens = content.split()
    total_tokens += len(tokens)

print(f"Total tokens (approximate): {total_tokens:,}")

# Optional: Display average tokens per document
avg_tokens = total_tokens / len(df)
print(f"Average tokens per document: {avg_tokens:.2f}")

In [ ]:
# Calculate token count for each document
df['token_count'] = df['content'].apply(lambda x: len(x.split()))

# Count entries with token length > 6000
long_docs_count = (df['token_count'] > 36000).sum()

# Get percentage of long documents
percentage_long = (long_docs_count / len(df)) * 100

# Show results
print(f"Number of documents with > 16000 tokens: {long_docs_count:,} ({percentage_long:.2f}%)")

In [6]:
cases = df[df['court'] == 'hkoat']

In [7]:
def process_cases(df, output_file="result"):
    cases = df.to_dict(orient='records')

    results = []

     # Process cases in parallel using ThreadPoolExecutor instead of ProcessPoolExecutor
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(generate_extra_meta, case["content"]) for case in cases]
        
        for future, text in tqdm(zip(as_completed(futures), cases), total=len(cases), desc="Processing cases"):
            try:
                data = future.result()
                results.append({
                    **data.model_dump(),
                    "case_name": text["case_name"],
                    "court": text["court"],
                    "date": text["date"].strftime("%Y-%m-%d"),
                    "citation": text["citation"],
                    "case_number": text["case_number"],
                    "law_report_citations": text["law_report_citations"],
                    "url": text["url"]
                })
                
                # Save intermediate results after each successful completion
                with open(f"{output_file}_partial.json", "w") as f:
                    f.write(json.dumps(results, indent=2))
            except Exception as e:
                print(f"Error processing case: {str(e)}")


    with open(f"{output_file}.json", "w") as f:
        f.write(json.dumps(results, indent=2))

In [ ]:
process_cases(cases)

INFO 03-26 20:21:53 [chat_utils.py:346] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
INFO 03-26 20:21:53 [chat_utils.py:346] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processing cases:   0%|          | 0/2 [00:00<?, ?it/s]


Processing cases:   0%|          | 0/2 [03:38<?, ?it/s]
